In [ ]:
# 기본 패키지
import time
import pandas as pd
from bs4 import BeautifulSoup

# 크롤링 패키지
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select

# NLP 패키지
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TextClassificationPipeline

In [ ]:
# chromedriver가 저장된 위치
driver_dir = "chromedriver.exe"

# 결과물을 저장할 위치
data_dir = ""

# 데이터를 수집 할 작품 제목
novel = "혁명은 내 취향이 아니었다"
# novel = "국정원 요원 천재배우로 환생"

driver = webdriver.Chrome(driver_dir) # 크롬드라이버 경로 설정

# url입력
url = "https://pagestage.kakao.com/search" # 사이트 입력
driver.get(url) # 사이트 오픈
time.sleep(5) # 5초 지연

In [ ]:
# 검색창 선택
xpath = '''//*[@id="__next"]/div[1]/div/main/div/div/div[1]/div/form/div/input'''
element = driver.find_element(by=By.XPATH, value=xpath)
element.clear()
time.sleep(2)

# 검색창 입력
element.send_keys(novel)
time.sleep(2)

element.send_keys(Keys.ENTER)
time.sleep(2)

# 처음 보이는 검색결과 클릭(19금 작품은 로그인 되어있지 않아 열람 불가능)
search_xpath = '''//*[@id="__next"]/div[1]/div/main/div/div/div[3]/div/div[2]/div[1]/div'''
driver.find_element(by=By.XPATH, value=search_xpath).click()
time.sleep(2)

In [4]:
# 더보기 버튼이 없어지기 전까지 무한클릭 -> 모든 댓글 확보
while True:
    try:
        more_xpath = '''//*[@id="__next"]/div[1]/div/main/section/div/div[3]/section/button'''
        driver.find_element(by=By.XPATH, value=more_xpath).click()
        time.sleep(3)
    except:
        break

# 스포일러 방지 버튼이 없어지기 전까지 무한클릭 -> 모든 내용 온전히 확보
while True:
    try:
        spoil_xpath = '''//*[@id="__next"]/div[1]/div/main/section/div/div[3]/section/div/div/div/p/button'''
        driver.find_element(by=By.XPATH, value=spoil_xpath).click()
        time.sleep(2)
    except:
        break

In [5]:
# 페이지 소스 출력
html = driver.page_source
html_source = BeautifulSoup(html, 'html.parser')
full_comments = html_source.find_all("div", attrs={"class":"flex-[0_1_100%] relative"})  # 전체 댓글 스크립트

In [6]:
# 검색 페이지로 돌아가기
driver.get(url)
time.sleep(5)

In [8]:
driver.close()  # 크롬드라이버 종료

In [6]:
# 데이터 추출
nicknames = []
dates = []
story_nums = []
sentences = []
likes = []

for comment in full_comments:
    nickname = comment.find("p", attrs={"class":"break-words font-bold mr-6"}).string
    date = comment.find("div", attrs={"class":"flex-[0_0_auto]"}).string
    story_num = comment.find("a").string
    sentence = comment.find("p", attrs={"class":"leading-[21px] break-all"}).string
    like = comment.find("span", attrs={"class":"font-gmarket font-medium leading-[normal] pt-[0.2em] text-12 text-13"}).string
    
    # 댓글 전처리 작업
    sentence = sentence.replace("#스테이지스테플러_대가성활동", "").strip()

    nicknames.append(nickname)
    dates.append(date)
    story_nums.append(story_num)
    sentences.append(sentence)
    likes.append(like)

In [ ]:
# 댓글 감정분석
model_name = "searle-j/kote_for_easygoing_people"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

pipe = TextClassificationPipeline(
        model=model,
        tokenizer=tokenizer,
        device=0, # gpu number, -1 if cpu used
        return_all_scores=True,
        function_to_apply='sigmoid'
    )

emotions = []

for outputs in pipe(sentences):
    temp = set()

    for output in outputs:
        if output["score"]>0.4:
            temp.add(output["label"])

    emotions.append(temp)

In [8]:
# csv 파일로 저장하기
total_comments = pd.DataFrame({'nickname':nicknames, 'date':dates,'story_num':story_nums,
                            'comment':sentences,'like':likes, 'emotion':emotions})
    
total_comments.to_csv(data_dir + f"comment_data.csv", index=False, encoding="utf-8")
